# Работа с декораторами, контекстными менеджерами

## Импорт модулей

In [1]:
from lesson26_task1 import role_required, USER
from lesson26_task2 import cache, get_info, CACHE
from lesson26_task3 import safe_write

## Задача 1. Права администратора

Проверяем работу декоратора.

In [2]:
@role_required("admin")
def secret_resource():
    """
    Function gives access to users with the 'admin' role.
    """
    return "Permission accepted"


USER["role"] = "admin"
print(f"user_role = {USER["role"]}: {secret_resource()}")

USER["role"] = "manager"
print(f"user_role = {USER["role"]}: {secret_resource()}")

USER["role"] = "developer"
print(f"user_role = {USER["role"]}: {secret_resource()}")

USER["role"] = "admin"
print(f"user_role = {USER["role"]}: {secret_resource()}")

user_role = admin: Permission accepted
user_role = manager: Permission denied
user_role = developer: Permission denied
user_role = admin: Permission accepted


## Задача 2. Кэширование

Декорируем функцию.

In [3]:
@cache("sql", 5)
def get_info_sql_5(thing):
    """
    Function returns information about 'thing' from SQL database
    with expiration time equals 5.
    """
    return get_info(thing)

Запускаем функцию с аргументом "bike".

In [4]:
for i in range(8):
    res = get_info_sql_5(thing="bike")
    print(res)

Info about: bike from sql, now cached with expire=5
bike info: 8319
Info about: bike cached in sql, expire=4
bike info: 8319
Info about: bike cached in sql, expire=3
bike info: 8319
Info about: bike cached in sql, expire=2
bike info: 8319
Info about: bike cached in sql, expire=1
bike info: 8319
Info about: bike cached in sql, expire=0
Caсhe has been cleared!
Info about: bike from sql, now cached with expire=5
bike info: 5179
Info about: bike cached in sql, expire=4
bike info: 5179


Запускаем функцию с аргументом "car".

In [5]:
for i in range(8):
    res = get_info_sql_5("car")
    print(res)

Info about: car from sql, now cached with expire=5
car info: 2520
Info about: car cached in sql, expire=4
car info: 2520
Info about: car cached in sql, expire=3
car info: 2520
Info about: car cached in sql, expire=2
car info: 2520
Info about: car cached in sql, expire=1
car info: 2520
Info about: car cached in sql, expire=0
Caсhe has been cleared!
Info about: car from sql, now cached with expire=5
car info: 9290
Info about: car cached in sql, expire=4
car info: 9290


Проверяем кэш.

In [6]:
for key in CACHE:
    print(f"{key}: {CACHE[key]}")

db=sql,expiration_time=5,thing=bike: ['bike info: 5179', 3]
db=sql,expiration_time=5,thing=car: ['car info: 9290', 3]


Продолжаем запускать функцию с аргументом "bike".

In [7]:
for i in range(8):
    res = get_info_sql_5(thing="bike")
    print(res)

Info about: bike cached in sql, expire=3
bike info: 5179
Info about: bike cached in sql, expire=2
bike info: 5179
Info about: bike cached in sql, expire=1
bike info: 5179
Info about: bike cached in sql, expire=0
Caсhe has been cleared!
Info about: bike from sql, now cached with expire=5
bike info: 7183
Info about: bike cached in sql, expire=4
bike info: 7183
Info about: bike cached in sql, expire=3
bike info: 7183
Info about: bike cached in sql, expire=2
bike info: 7183


Продолжаем запускать функцию с аргументом "car".

In [8]:
for i in range(8):
    res = get_info_sql_5("car")
    print(res)

Info about: car cached in sql, expire=3
car info: 9290
Info about: car cached in sql, expire=2
car info: 9290
Info about: car cached in sql, expire=1
car info: 9290
Info about: car cached in sql, expire=0
Caсhe has been cleared!
Info about: car from sql, now cached with expire=5
car info: 3070
Info about: car cached in sql, expire=4
car info: 3070
Info about: car cached in sql, expire=3
car info: 3070
Info about: car cached in sql, expire=2
car info: 3070


Еще раз проверяем кэш.

In [9]:
for key in CACHE:
    print(f"{key}: {CACHE[key]}")

db=sql,expiration_time=5,thing=bike: ['bike info: 7183', 1]
db=sql,expiration_time=5,thing=car: ['car info: 3070', 1]


Декорируем функции с другими параметрами.

In [10]:
@cache("postgresql", 5)
def get_info_postgresql_5(thing):
    """
    Function returns information about 'thing' from postgresql database
    with expiration time equals 5.
    """
    return get_info(thing)


@cache("sqlite", 3)
def get_info_sqlite_3(thing):
    """
    Function returns information about 'thing' from sqlite database
    with expiration time equals 3.
    """
    return get_info(thing)

Проверяем работу.

In [11]:
for i in range(7):
    get_info_postgresql_5("bike_store")

for i in range(5):
    get_info_sqlite_3("bike_store")

print()

for i in range(7):
    get_info_postgresql_5("users")

for i in range(5):
    get_info_sqlite_3("users")

Info about: bike_store from postgresql, now cached with expire=5
Info about: bike_store cached in postgresql, expire=4
Info about: bike_store cached in postgresql, expire=3
Info about: bike_store cached in postgresql, expire=2
Info about: bike_store cached in postgresql, expire=1
Info about: bike_store cached in postgresql, expire=0
Info about: bike_store from postgresql, now cached with expire=5
Info about: bike_store from sqlite, now cached with expire=3
Info about: bike_store cached in sqlite, expire=2
Info about: bike_store cached in sqlite, expire=1
Info about: bike_store cached in sqlite, expire=0
Info about: bike_store from sqlite, now cached with expire=3

Info about: users from postgresql, now cached with expire=5
Info about: users cached in postgresql, expire=4
Info about: users cached in postgresql, expire=3
Info about: users cached in postgresql, expire=2
Info about: users cached in postgresql, expire=1
Info about: users cached in postgresql, expire=0
Info about: users from

## Задача 3. Контекстный менеджер

Проверяем работу контекстного менеджера.

In [12]:
with safe_write("lesson26_task3_test1.txt") as file:
    file.write("Я знаю, что ничего не знаю, но другие не знают и этого.")

with open("lesson26_task3_test1.txt", encoding="utf-8") as file:
    print(file.read())

Я знаю, что ничего не знаю, но другие не знают и этого.


In [13]:
with safe_write("lesson26_task3_test2.txt") as file:
    file.write("Я знаю, что ничего не знаю, но другие не знают и этого. \n")

with safe_write("lesson26_task3_test2.txt") as file:
    print(
        "Если ты будешь любознательным, то будешь много знающим.", file=file, flush=True
    )
    raise ValueError

with open("lesson26_task3_test2.txt", encoding="utf-8") as file:
    print(file.read())

Во время записи в файл было возбуждено исключение ValueError
Я знаю, что ничего не знаю, но другие не знают и этого. 

